# Machine Learning 2020 Course Projects

# Notebook of Team Wok: Cristof Rojas, David Föger

In [1]:
# Quick load dataset and check
import pandas as pd
import numpy as np
import pickle
import time

from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.impute import SimpleImputer


from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE


"""
import seaborn as sns

from itertools import takewhile

from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing
from sklearn import svm
from sklearn.decomposition import PCA
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

from imblearn.under_sampling import ClusterCentroids
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import NearMiss
from imblearn.combine import SMOTETomek

ignore some of the warnings of sklearn 
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
"""



!pip install --upgrade category_encoders

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


     |████████████████████████████████| 81kB 2.2MB/s 


In [2]:
filename = "train_set.csv"
data_train = pd.read_csv(filename)
filename = "test_set.csv"
data_test = pd.read_csv(filename)

In [3]:
data_train.describe()

,id,target,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,ps_ind_10_bin,ps_ind_11_bin,ps_ind_12_bin,ps_ind_13_bin,ps_ind_14,ps_ind_15,ps_ind_16_bin,ps_ind_17_bin,ps_ind_18_bin,ps_reg_01,ps_reg_02,ps_reg_03,ps_car_01_cat,ps_car_02_cat,ps_car_03_cat,ps_car_04_cat,ps_car_05_cat,ps_car_06_cat,ps_car_07_cat,ps_car_08_cat,ps_car_09_cat,ps_car_10_cat,ps_car_11_cat,ps_car_11,ps_car_12,ps_car_13,ps_car_14,ps_car_15,ps_calc_01,ps_calc_02,ps_calc_03,ps_calc_04,ps_calc_05,ps_calc_06,ps_calc_07,ps_calc_08,ps_calc_09,ps_calc_10,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
count,4.464100e+05,446410.000000,446410.000000,446410.000000,446410.000000,446410.000000,446410.000000,446410.000000,446410.000000,446410.000000,446410.000000,446410.000000,446410.000000,446410.000000,446410.000000,446410.000000,446410.000000,446410.000000,446410.000000,446410.000000,446410.000000,446410.000000,446410.000000,446410.000000,446410.000000,446410.000000,446410.000000,446410.000000,446410.000000,446410.000000,446410.000000,446410.000000,446410.000000,446410.000000,446410.000000,446410.000000,446410.000000,446410.000000,446410.000000,446410.000000,446410.000000,446410.000000,446410.000000,446410.000000,446410.000000,446410.000000,446410.000000,446410.000000,446410.000000,446410.000000,446410.000000,446410.000000,446410.000000,446410.000000,446410.000000,446410.000000,446410.000000,446410.000000,446410.000000
mean,5.578496e+05,0.036449,1.898952,1.359105,4.424744,0.417054,0.405022,0.393829,0.256701,0.164047,0.185424,0.000394,0.001698,0.009406,0.000983,0.012482,7.303091,0.661101,0.120828,0.153307,0.611023,0.439810,0.551161,8.294812,0.830252,-0.504861,0.721048,-0.157673,6.557136,0.910461,0.831919,1.329477,0.992151,62.196306,2.345331,0.379889,0.813047,0.276317,3.065864,0.450049,0.449649,0.449828,2.369951,1.885919,7.688573,3.006873,9.225795,2.339056,8.435570,5.441536,1.442483,2.872330,7.538823,0.122685,0.627835,0.554569,0.287762,0.348948,0.153668
std,3.219297e+05,0.187404,1.984344,0.665016,2.702142,0.493345,1.349971,0.488598,0.436814,0.370318,0.388641,0.019852,0.041172,0.096528,0.031344,0.128162,3.545933,0.473336,0.325928,0.360284,0.287833,0.404693,0.794362,2.509490,0.375423,0.788453,2.147046,0.844358,5.500921,0.346083,0.373939,0.978255,0.091513,33.001035,0.832321,0.058320,0.224126,0.356915,0.732508,0.287028,0.286930,0.287122,1.117123,1.135501,1.334470,1.414191,1.460074,1.246988,2.906923,2.332302,1.203418,1.693303,2.746262,0.328076,0.483382,0.497014,0.452720,0.476638,0.360631
min,7.000000e+00,0.000000,0.000000,-1.000000,0.000000,-1.000000,-1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.000000,-1.000000,-1.000000,-1.000000,0.000000,-1.000000,0.000000,-1.000000,0.000000,-1.000000,0.000000,1.000000,-1.000000,-1.000000,0.250619,-1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.793988e+05,0.000000,0.000000,1.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,0.000000,0.000000,0.000000,0.400000,0.200000,0.525000,7.000000,1.000000,-1.000000,0.000000,-1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,32.000000,2.000000,0.316228,0.670904,0.333167,2.828427,0.200000,0.200000,0.200000,2.000000,1.000000,7.000000,2.000000,8.000000,1.000000,6.000000,4.000000,1.000000,2.000000,6.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,5.576260e+05,0.000000,1.000000,1.000000,4.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,7.000000,1.000000,0.000000,0.000000,0.700000,0.300000,0.720677,7.000000,1.000000,-1.000000,0.000000,0.000000,7.0000

In [4]:
data_test.describe()

,id,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,ps_ind_10_bin,ps_ind_11_bin,ps_ind_12_bin,ps_ind_13_bin,ps_ind_14,ps_ind_15,ps_ind_16_bin,ps_ind_17_bin,ps_ind_18_bin,ps_reg_01,ps_reg_02,ps_reg_03,ps_car_01_cat,ps_car_02_cat,ps_car_03_cat,ps_car_04_cat,ps_car_05_cat,ps_car_06_cat,ps_car_07_cat,ps_car_08_cat,ps_car_09_cat,ps_car_10_cat,ps_car_11_cat,ps_car_11,ps_car_12,ps_car_13,ps_car_14,ps_car_15,ps_calc_01,ps_calc_02,ps_calc_03,ps_calc_04,ps_calc_05,ps_calc_06,ps_calc_07,ps_calc_08,ps_calc_09,ps_calc_10,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
count,148800.000000,148800.000000,148800.000000,148800.000000,148800.000000,148800.000000,148800.000000,148800.000000,148800.000000,148800.000000,148800.000000,148800.000000,148800.000000,148800.000000,148800.000000,148800.000000,148800.000000,148800.000000,148800.000000,148800.000000,148800.000000,148800.000000,148800.000000,148800.000000,148800.000000,148800.000000,148800.000000,148800.000000,148800.000000,148800.000000,148800.000000,148800.000000,148800.000000,148800.000000,148800.000000,148800.000000,148800.000000,148800.000000,148800.000000,148800.000000,148800.000000,148800.000000,148800.000000,148800.000000,148800.000000,148800.00000,148800.000000,148800.000000,148800.000000,148800.000000,148800.000000,148800.000000,148800.000000,148800.000000,148800.000000,148800.000000,148800.000000,148800.000000
mean,174399.500000,1.904651,1.358461,4.419032,0.416022,0.405692,0.393474,0.258031,0.163548,0.184946,0.000309,0.001673,0.009536,0.000840,0.012359,7.290376,0.659987,0.121841,0.153864,0.610895,0.437308,0.550922,8.299261,0.828965,-0.505007,0.737634,-0.157897,6.549966,0.908723,0.832560,1.327117,0.992090,62.273508,2.348286,0.380114,0.813919,0.276074,3.066007,0.448878,0.449411,0.449913,2.378454,1.885793,7.692070,3.002668,9.22623,2.338978,8.427675,5.440887,1.440228,2.872157,7.539657,0.121647,0.627863,0.553024,0.285444,0.349254,0.152272
std,42955.004365,1.982130,0.663332,2.693171,0.493212,1.352667,0.488522,0.437553,0.369867,0.388255,0.017580,0.040873,0.097188,0.028972,0.125678,3.546365,0.473715,0.327104,0.360820,0.287072,0.402974,0.790939,2.504612,0.376594,0.789263,2.172572,0.844598,5.503029,0.350157,0.373369,0.980226,0.091936,33.046788,0.833230,0.058347,0.225969,0.357874,0.727937,0.287708,0.286782,0.287249,1.117472,1.133212,1.333845,1.415691,1.45847,1.246839,2.897621,2.334537,1.201600,1.699644,2.747838,0.326878,0.483376,0.497182,0.451627,0.476736,0.359285
min,100000.000000,0.000000,-1.000000,0.000000,-1.000000,-1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.000000,-1.000000,-1.000000,-1.000000,0.000000,-1.000000,0.000000,-1.000000,0.000000,-1.000000,0.000000,1.000000,-1.000000,0.100000,0.287336,-1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,2.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,137199.750000,0.000000,1.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,0.000000,0.000000,0.000000,0.400000,0.200000,0.525595,7.000000,1.000000,-1.000000,0.000000,-1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,32.000000,2.000000,0.316228,0.670420,0.334066,2.828427,0.200000,0.200000,0.200000,2.000000,1.000000,7.000000,2.000000,8.00000,1.000000,6.000000,4.000000,1.000000,2.000000,6.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,174399.500000,1.000000,1.000000,4.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,7.000000,1.000000,0.000000,0.000000,0.700000,0.300000,0.720243,7.000000,1.000000,-1.000000,0.000000,0.000000,7.000000,1.000000,1.000000,2.000000,1.000000,65.000000,3.000000,0.374166,

In [5]:
## Column
def get_all_elements(data_train_median):
    cols = data_train.columns[1:]
    thisdict = dict(zip(cols,list(map(lambda x: list(set(x)),\
                                      data_train.values[:,1:].T))))
    return thisdict

thisdict = get_all_elements(data_train)

feature_cols = data_train.columns[2:]
tokens = list(map(lambda col: col.split("_"), feature_cols))

postfix = list(map(lambda toks: toks[-1], tokens))
postfix = [ele for ele in postfix if not ele.isdigit()]
surfix = list(map(lambda toks: toks[1], tokens))

postfix_cnt = [postfix.count(ele) for ele in set(postfix)]
surfix_cnt = [surfix.count(ele) for ele in set(surfix)]

# Get all large variation variable
small_col = [x[0] for x in list(thisdict.items()) if len(x[1]) < 11]

def get_dist(df): 
    # Creating a dictionary for every passenger class count in every deck
    features = df.columns.levels[0]
    pclasses = df.columns.levels[1]
    num_f_1 = len(features)
    num_f_2 = len(pclasses)
    counts = np.zeros((num_f_1,num_f_2))
    for i, feature in enumerate(features):
        for j, pclass in enumerate(pclasses):
            try:
                count = df[feature][pclass][0]
                counts[i][j] = count 
            except KeyError:
                counts[i][j] = 0
                
    counts_sum = np.sum(counts.T, axis=0)
    f_percentages = 100 * counts.T/counts_sum
    return counts, f_percentages


## Check the hit rate from different cat values
## Get all category data
# Get all large variation variable

In [6]:
# MISSING VALUES
data_train_more = data_train.copy()
np.sum(data_train_more.values == -1) ## Detect if there is -1 in the dataset
bad_feature = [ele[0] for ele in list(thisdict.items()) if -1 in ele[1] ]
bad_feature_alt = [col for col in data_train.columns
                     if -1 in data_train[col].values]

imputer = SimpleImputer(missing_values=-1, strategy='median')
data_train_median = pd.DataFrame(imputer.fit_transform(data_train),
                              index = data_train.index,
                              columns = data_train.columns)

thisdict_full = get_all_elements(data_train_median)

large_col = [x[0] for x in list(thisdict_full.items()) if len(x[1]) > 25]

tar = data_train['target'] == 1
feature = large_col[4]                     

data_train_median_flag = data_train_median.copy()
for col in bad_feature:
    misflags = data_train[col] == -1
    data_train_median_flag[col + '_was_missing'] = misflags.astype(int)
    
data_train_median_flag.iloc[:,-9:-1]

,ps_car_01_cat_was_missing,ps_car_02_cat_was_missing,ps_car_03_cat_was_missing,ps_car_05_cat_was_missing,ps_car_07_cat_was_missing,ps_car_09_cat_was_missing,ps_car_11_was_missing,ps_car_12_was_missing
0,0,0,1,0,0,0,0,0
1,0,0,1,1,0,0,0,0
2,0,0,1,1,0,0,0,0
3,0,0,0,0,0,0,0,0
4,0,0,1,1,0,0,0,0
...,...,...,...,...,...,...,...,...
446405,0,0,1,0,0,0,0,0
446406,0,0,1,1,0,0,0,0
446407,0,0,0,0,0,0,0,0
446408,0,0,1,0,0,0,0,0


In [7]:
## Correlation
inspect_feature = bad_feature[3]
df_all_corr = data_train.corr().abs().unstack().sort_values( \
                    kind="quicksort", ascending=False).reset_index()
df_all_corr.rename(columns={"level_0": "Feature 1", 
                            "level_1": "Feature 2", 
                            0: 'Correlation Coefficient'}, 
                   inplace=True)
most_related = df_all_corr[df_all_corr['Feature 1'] == inspect_feature]['Feature 2'].iloc[1]
pd.set_option('display.max_rows', 10)
df_all_corr[df_all_corr['Feature 1'] == inspect_feature]

data_train_group = data_train.copy()
data_train_group[inspect_feature] = data_train_group[inspect_feature].replace(-1, np.nan)
data_train_group[inspect_feature] = data_train_group.groupby([most_related])[inspect_feature].apply(lambda x: x.fillna(x.mean()))
np.sum(data_train_group[inspect_feature] == -1)
data_train_group[[inspect_feature]].join(data_train[[inspect_feature]],lsuffix='_filled', rsuffix='_old')

,ps_reg_03_filled,ps_reg_03_old
0,0.920598,0.920598
1,1.299519,1.299519
2,1.586860,1.586860
3,0.875357,0.875357
4,1.477117,1.477117
...,...,...
446405,0.574456,0.574456
446406,0.756471,-1.000000
446407,0.653554,-1.000000
446408,1.118034,1.118034


In [8]:
## Feature Engineering
## Turn continuous to discrete variable

pd.reset_option('display.max_rows')
feature = large_col[4]
data_train_en = data_train.copy()
data_train_en[feature] = pd.qcut(data_train[feature], 10)

In [9]:
## Combined Features

data_train_en['new'] = data_train_en['ps_ind_14'] + data_train_en['ps_ind_15'] + 1

ind_map = {1: 'Small', 2: 'Small', 3: 'Small', 4: 'Small', 5: 'Medium', 6: 'Medium', 7: 'Large', 8: 'Large', 11: 'Large',13: 'Large',14: 'Large',15: 'Large'}
data_train_en['Grouped'] = data_train_en['new'].map(ind_map)
data_train_en['new']

0          6
1          4
2          9
3         14
4         12
          ..
446405     6
446406    12
446407    10
446408     8
446409    11
Name: new, Length: 446410, dtype: int64

In [10]:
## Category data transformation

data_train_en['ps_ind_14'] = data_train_en.groupby('ps_ind_14')['ps_ind_14'].transform('count')
import category_encoders as ce
cat_features = small_col[2:4]
count_enc = ce.CountEncoder()
count_encoded = count_enc.fit_transform(data_train_en[cat_features])

data = data_train_en.join(count_encoded.add_suffix("_count"))

# Target Encoding

target_enc = ce.TargetEncoder(cols=cat_features)
## Split the dataset first!!!
target_enc.fit(data_train_en[cat_features], data_train_en['target'])
target_enc.fit(data_train_en[cat_features], data_train_en['target'])
traindata = data_train_en.join(target_enc.transform(data_train_en[cat_features]).add_suffix('_target'))


## Feature Transformation
pd.get_dummies(traindata, columns=["ps_ind_02_cat"]).head()
pd.get_dummies(traindata, columns=["ps_ind_02_cat", "ps_ind_04_cat"], prefix=["a", "b"]).head()

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


,id,target,ps_ind_01,ps_ind_03,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,ps_ind_10_bin,ps_ind_11_bin,ps_ind_12_bin,ps_ind_13_bin,ps_ind_14,ps_ind_15,ps_ind_16_bin,ps_ind_17_bin,ps_ind_18_bin,ps_reg_01,ps_reg_02,ps_reg_03,ps_car_01_cat,ps_car_02_cat,ps_car_03_cat,ps_car_04_cat,ps_car_05_cat,ps_car_06_cat,ps_car_07_cat,ps_car_08_cat,ps_car_09_cat,ps_car_10_cat,ps_car_11_cat,ps_car_11,ps_car_12,ps_car_13,ps_car_14,ps_car_15,ps_calc_01,ps_calc_02,ps_calc_03,ps_calc_04,ps_calc_05,ps_calc_06,ps_calc_07,ps_calc_08,ps_calc_09,ps_calc_10,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin,new,Grouped,ps_ind_02_cat_target,ps_ind_04_cat_target,a_-1,a_1,a_2,a_3,a_4,b_-1,b_0,b_1
0,818751,1,3,5,0,0,1,0,0,0,0,0,0,441630,5,1,0,0,0.3,0.0,0.920598,4,1,-1,9,0,15,1,1,0,1,41,2,0.519615,1.376287,"(0.423, 0.636]",3.464102,0.2,0.4,0.7,1,2,8,2,12,2,15,5,2,2,7,0,1,0,0,1,0,6,Medium,0.035633,0.034666,0,1,0,0,0,0,1,0
1,472508,0,3,7,0,0,1,0,0,0,0,0,0,441630,3,0,0,1,0.9,1.8,1.299519,11,0,-1,0,-1,15,1,1,1,1,104,2,0.447214,1.167837,"(0.391, 0.402]",3.605551,0.2,0.9,0.1,2,3,10,3,9,2,7,6,2,1,5,0,1,0,1,0,0,4,Small,0.037933,0.038822,0,0,0,1,0,0,0,1
2,688265,0,0,0,0,0,0,0,1,0,0,0,0,441630,8,1,0,0,0.8,1.0,1.586860,11,0,-1,0,-1,7,1,1,0,1,92,3,0.400000,0.830039,"(0.402, 0.423]",3.316625,0.6,0.5,0.8,3,3,6,1,9,4,7,3,0,1,7,0,0,0,0,0,0,9,NaN,0.038175,0.038822,0,0,1,0,0,0,0,1
3,236368,0,1,5,2,0,1,0,0,0,0,0,0,441630,13,1,0,0,0.4,0.3,0.875357,11,0,0,9,0,15,1,1,1,1,41,2,0.547723,1.504799,"(0.423, 0.636]",3.316625,0.4,0.4,0.9,1,2,8,2,9,2,8,12,2,1,9,0,1,0,0,1,0,14,Large,0.035633,0.038822,0,1,0,0,0,0,0,1
4,34973,0,5,5,0,0,0,0,1,0,0,0,0,441630,11,1,0,0,0.9,0.9,1.477117,11,1,-1,0,-1,0,1,1,2,1,87,3,0.316228,0.668383,"(0.349, 0.362]",3.000000,0.0,0.6,0.1,4,2,8,2,7,4,11,3,4,1,5,0,1,1,0,1,1,12,NaN,0.037933,0.038822,0,0,0,1,0,0,0,1


In [11]:
## Feature Selection

## Univariate Selection
from sklearn.feature_selection import SelectKBest, chi2, f_classif, mutual_info_regression
feature_cols = data_train_median_flag.columns[2:]
selector = SelectKBest(f_classif, k=35)
data_new = selector.fit_transform(data_train_median_flag[feature_cols], data_train_median_flag['target'])

selected_features = pd.DataFrame(selector.inverse_transform(data_new), 
                                 index=data_train_median_flag.index, 
                                 columns=feature_cols)
selected_columns = selected_features.columns[selected_features.var() != 0]

In [12]:
y_true = data_train_median[['target']]
data_f_score = data_train_median.drop(columns=['id', 'target'])

## Select target and features
feature_cols = data_train_median.columns[2:]
data_Y = data_train_median['target']
data_X = data_train_median[feature_cols]

data_train_median.describe()

,id,target,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,ps_ind_10_bin,ps_ind_11_bin,ps_ind_12_bin,ps_ind_13_bin,ps_ind_14,ps_ind_15,ps_ind_16_bin,ps_ind_17_bin,ps_ind_18_bin,ps_reg_01,ps_reg_02,ps_reg_03,ps_car_01_cat,ps_car_02_cat,ps_car_03_cat,ps_car_04_cat,ps_car_05_cat,ps_car_06_cat,ps_car_07_cat,ps_car_08_cat,ps_car_09_cat,ps_car_10_cat,ps_car_11_cat,ps_car_11,ps_car_12,ps_car_13,ps_car_14,ps_car_15,ps_calc_01,ps_calc_02,ps_calc_03,ps_calc_04,ps_calc_05,ps_calc_06,ps_calc_07,ps_calc_08,ps_calc_09,ps_calc_10,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
count,4.464100e+05,446410.000000,446410.000000,446410.000000,446410.000000,446410.000000,446410.000000,446410.000000,446410.000000,446410.000000,446410.000000,446410.000000,446410.000000,446410.000000,446410.000000,446410.000000,446410.000000,446410.000000,446410.000000,446410.000000,446410.000000,446410.000000,446410.000000,446410.000000,446410.000000,446410.000000,446410.000000,446410.000000,446410.000000,446410.000000,446410.000000,446410.000000,446410.000000,446410.000000,446410.000000,446410.000000,446410.000000,446410.000000,446410.000000,446410.000000,446410.000000,446410.000000,446410.000000,446410.000000,446410.000000,446410.000000,446410.000000,446410.000000,446410.000000,446410.000000,446410.000000,446410.000000,446410.000000,446410.000000,446410.000000,446410.000000,446410.000000,446410.000000,446410.000000
mean,5.578496e+05,0.036449,1.898952,1.359799,4.424744,0.417188,0.414798,0.393829,0.256701,0.164047,0.185424,0.000394,0.001698,0.009406,0.000983,0.012482,7.303091,0.661101,0.120828,0.153307,0.611023,0.439810,0.878009,8.296228,0.830261,0.876542,0.721048,0.737799,6.557136,0.948713,0.831919,1.332286,0.992151,62.196306,2.345357,0.379892,0.813047,0.374536,3.065864,0.450049,0.449649,0.449828,2.369951,1.885919,7.688573,3.006873,9.225795,2.339056,8.435570,5.441536,1.442483,2.872330,7.538823,0.122685,0.627835,0.554569,0.287762,0.348948,0.153668
std,3.219297e+05,0.187404,1.984344,0.663595,2.702142,0.493095,1.343366,0.488598,0.436814,0.370318,0.388641,0.019852,0.041172,0.096528,0.031344,0.128162,3.545933,0.473336,0.325928,0.360284,0.287833,0.404693,0.315092,2.506500,0.375403,0.328963,2.147046,0.439832,5.500921,0.220583,0.373939,0.975866,0.091513,33.001035,0.832277,0.058284,0.224126,0.043894,0.732508,0.287028,0.286930,0.287122,1.117123,1.135501,1.334470,1.414191,1.460074,1.246988,2.906923,2.332302,1.203418,1.693303,2.746262,0.328076,0.483382,0.497014,0.452720,0.476638,0.360631
min,7.000000e+00,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.061237,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.141421,0.250619,0.109545,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.793988e+05,0.000000,0.000000,1.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,0.000000,0.000000,0.000000,0.400000,0.200000,0.666615,7.000000,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,1.000000,32.000000,2.000000,0.316228,0.670904,0.353553,2.828427,0.200000,0.200000,0.200000,2.000000,1.000000,7.000000,2.000000,8.000000,1.000000,6.000000,4.000000,1.000000,2.000000,6.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,5.576260e+05,0.000000,1.000000,1.000000,4.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,7.000000,1.000000,0.000000,0.000000,0.700000,0.300000,0.802340,7.000000,1.000000,1.000000,0.000000,1.000000,7.000000,1.000000,1.0000

In [13]:
# the data is very unbalanced
def balancing_problem(data_Y):
    target_count = data_Y.value_counts()
    print('Balancing of the target data: ')
    print('zeros:', target_count[0])
    print('ones:', target_count[1])
    print('zeros to ones:', round(target_count[0] / target_count[1], 2), ': 1')
    print(round(target_count[0] * 100 / (target_count[0] + target_count[1]), 2), '% are zeros')
    print(round(target_count[1] * 100 / (target_count[0] + target_count[1]), 2), '% are ones')

balancing_problem(data_Y)    

Balancing of the target data: 
zeros: 430139
ones: 16271
zeros to ones: 26.44 : 1
96.36 % are zeros
3.64 % are ones


In [14]:
# old code of phase 1
"""
#pca = PCA(n_components=2)
#X = pca.fit_transform(x_train)

#sample = ClusterCentroids(random_state=42, sampling_strategy='auto', n_jobs=-1)
#X, Y = sample.fit_resample(x_train, y_train)

#sample = SMOTE()
#X, Y = sample.fit_resample(x_train, y_train)

#model = svm.SVC(kernel='linear')
#model.fit(x_train, y_train)

#x_scaled = preprocessing.scale(X)
#model = LogisticRegression()
#model.fit(x_scaled, Y)

best = 0
# save the starting time of the overall mode 
overall_time = time.clock()

for i in range(1):
    # save the starting time of every iteration
    start_time = time.clock()
    # split the data in train and test data
    x_train, x_val, y_train, y_val = train_test_split(data_X, data_Y, test_size = 0.01, shuffle = True)  

    # create a randomsampler with "majority" strategy
    # RandomUnderSampler(sampling_strategy='majority', replacement=False, random_state=15000)
    sampler = RandomUnderSampler(sampling_strategy='majority', replacement=True, random_state=15000)
    X, Y = sampler.fit_resample(x_train, y_train)

    # use a scaler for the svc
    X_svc = StandardScaler().fit(X).transform(X)

    #create an svc model and fit the under sampled data
    #class_weight='balanced', C=100, dual=False, max_iter=15000
    #max_iter=2000
    model = LinearSVC(max_iter=2000)
    #X_svc
    model.fit(X, Y)

    # calculate the f score of the predicted data
    y_predict = model.predict(data_f_score)
    score = f1_score(y_true, y_predict, average='macro')
    
    # check if the new f score is better than the old one
    # and save it to the pickle file
    print("iteration:          ", i)
    if score > best:
        best = score
        #print("iteration:          ", i)
        print("time of iteration: %6.2f minutes" % ((time.clock() - start_time) / 60))
        print("best fscore macro:   %6.5f" % best)
        print(" ")

        with open("submit.pickle", "wb") as f:
            pickle.dump(model, f)


# print the overall running time
print(" ")
print(" ")
print(" ")
print("overall time: %6.2f minutes" % ((time.clock() - overall_time) / 60))

# get the best model from the pickle file
pickle_in = open("submit.pickle", "rb")
model = pickle.load(pickle_in)

# print the best macro f-score
y_predict = model.predict(data_f_score)
score = f1_score(y_true, y_predict, average='macro')
print("fscore macro at the end: %6.5f" % best)
"""

'\n#pca = PCA(n_components=2)\n#X = pca.fit_transform(x_train)\n\n#sample = ClusterCentroids(random_state=42, sampling_strategy=\'auto\', n_jobs=-1)\n#X, Y = sample.fit_resample(x_train, y_train)\n\n#sample = SMOTE()\n#X, Y = sample.fit_resample(x_train, y_train)\n\n#model = svm.SVC(kernel=\'linear\')\n#model.fit(x_train, y_train)\n\n#x_scaled = preprocessing.scale(X)\n#model = LogisticRegression()\n#model.fit(x_scaled, Y)\n\nbest = 0\n# save the starting time of the overall mode \noverall_time = time.clock()\n\nfor i in range(1):\n    # save the starting time of every iteration\n    start_time = time.clock()\n    # split the data in train and test data\n    x_train, x_val, y_train, y_val = train_test_split(data_X, data_Y, test_size = 0.01, shuffle = True)  \n\n    # create a randomsampler with "majority" strategy\n    # RandomUnderSampler(sampling_strategy=\'majority\', replacement=False, random_state=15000)\n    sampler = RandomUnderSampler(sampling_strategy=\'majority\', replacement

In [ ]:
best = 0
overall_time = time.clock()
f_score = 0

for i in range(3):
  # split the data in train and test data
  x_train, x_val, y_train, y_val = train_test_split(data_X, data_Y, test_size = 0.1, shuffle = True) 

  # Resample the minority class. SMOTE(sampling_strategy='minority', random_state=42)     RandomUnderSampler(sampling_strategy='majority', replacement=True, random_state=15000)
  sampler = SMOTE(sampling_strategy='minority', random_state=42, k_neighbors=10) 
  x_train, y_train = sampler.fit_resample(x_train, y_train)

  model = RandomForestClassifier(n_estimators = 200, 
                               n_jobs=-1, 
                               class_weight='balanced', 
                               min_samples_leaf=50, 
                               min_samples_split=50, 
                               verbose=5)
  
  model.fit(x_train, y_train.ravel())

  # calculate the f-score of the training data
  y_predict = model.predict(x_val)
  f_score = f1_score(y_val, y_predict, average='macro')

  score = model.score(x_val, y_val)

  print("\n\n\n\n--------------------------------------")
  # check if the f-score got better and save the model if it did
  if f_score > best:
      best = f_score
      
      print("Best results so far on iteration", i, ": ")
      print("best fscore macro:   %6.5f" % f_score)
      print("best score: ", score)

      with open("submit.pickle", "wb") as f:
          pickle.dump(model, f)

  print("--------------------------------------\n\n\n\n")


# get the best model from the pickle file
pickle_in = open("submit.pickle", "rb")
model = pickle.load(pickle_in)


y_predict = model.predict(data_f_score)
f_score_after = f1_score(y_true, y_predict, average='macro')
score = model.score(data_f_score, y_true)


# print the overall running time
print("\n\n\n\n--------------------------------------")
print("Scores at the end after %6.2f minutes: " % ((time.clock() - overall_time) / 60))
print("fscore macro of training data: %6.5f" % f_score) 
print("fscore macro of all data:      %6.5f" % f_score_after)   
print("score:                        ", score)
print("confustion matrix: ")
print(confusion_matrix(y_true, y_predict))

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.


building tree 1 of 200
building tree 2 of 200
building tree 3 of 200
building tree 4 of 200
building tree 5 of 200
building tree 6 of 200
building tree 7 of 200
building tree 8 of 200
building tree 9 of 200
building tree 10 of 200
building tree 11 of 200
building tree 12 of 200
building tree 13 of 200
building tree 14 of 200
building tree 15 of 200
building tree 16 of 200


[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:   12.3s


building tree 17 of 200
building tree 18 of 200
building tree 19 of 200
building tree 20 of 200
building tree 21 of 200
building tree 22 of 200
building tree 23 of 200
building tree 24 of 200
building tree 25 of 200
building tree 26 of 200
building tree 27 of 200
building tree 28 of 200
building tree 29 of 200
building tree 30 of 200
building tree 31 of 200
building tree 32 of 200
building tree 33 of 200
building tree 34 of 200
building tree 35 of 200
building tree 36 of 200
building tree 37 of 200
building tree 38 of 200
building tree 39 of 200
building tree 40 of 200
building tree 41 of 200
building tree 42 of 200
building tree 43 of 200
building tree 44 of 200
building tree 45 of 200
building tree 46 of 200
building tree 47 of 200
building tree 48 of 200
building tree 49 of 200
building tree 50 of 200
building tree 51 of 200
building tree 52 of 200
building tree 53 of 200
building tree 54 of 200
building tree 55 of 200
building tree 56 of 200
building tree 57 of 200
building tree 58

[Parallel(n_jobs=-1)]: Done  68 tasks      | elapsed:   55.3s


...

building tree 17 of 200
building tree 18 of 200
building tree 19 of 200
building tree 20 of 200
building tree 21 of 200
building tree 22 of 200
building tree 23 of 200
building tree 24 of 200
building tree 25 of 200
building tree 26 of 200
building tree 27 of 200
building tree 28 of 200
building tree 29 of 200
building tree 30 of 200
building tree 31 of 200
building tree 32 of 200
building tree 33 of 200
building tree 34 of 200
building tree 35 of 200
building tree 36 of 200
building tree 37 of 200
building tree 38 of 200
building tree 39 of 200
building tree 40 of 200
building tree 41 of 200
building tree 42 of 200
building tree 43 of 200
building tree 44 of 200
building tree 45 of 200
building tree 46 of 200
building tree 47 of 200
building tree 48 of 200
building tree 49 of 200
building tree 50 of 200
building tree 51 of 200
building tree 52 of 200
building tree 53 of 200
building tree 54 of 200
building tree 55 of 200
building tree 56 of 200
building tree 57 of 200
building tree 58

[Parallel(n_jobs=-1)]: Done  68 tasks      | elapsed:   54.4s


building tree 70 of 200
building tree 71 of 200
building tree 72 of 200
building tree 73 of 200
building tree 74 of 200
building tree 75 of 200
building tree 76 of 200
building tree 77 of 200
building tree 78 of 200
building tree 79 of 200
building tree 80 of 200
building tree 81 of 200
building tree 82 of 200
building tree 83 of 200
building tree 84 of 200
building tree 85 of 200
building tree 86 of 200
building tree 87 of 200
building tree 88 of 200
building tree 89 of 200
building tree 90 of 200
building tree 91 of 200
building tree 92 of 200
building tree 93 of 200
building tree 94 of 200
building tree 95 of 200
building tree 96 of 200
building tree 97 of 200
building tree 98 of 200
building tree 99 of 200
building tree 100 of 200
building tree 101 of 200
building tree 102 of 200
building tree 103 of 200
building tree 104 of 200
building tree 105 of 200
building tree 106 of 200
building tree 107 of 200
building tree 108 of 200
building tree 109 of 200
building tree 110 of 200
build

[Parallel(n_jobs=-1)]: Done 158 tasks      | elapsed:  2.1min


building tree 160 of 200
building tree 161 of 200
building tree 162 of 200
building tree 163 of 200
building tree 164 of 200
building tree 165 of 200
building tree 166 of 200
building tree 167 of 200
building tree 168 of 200
building tree 169 of 200
building tree 170 of 200
building tree 171 of 200
building tree 172 of 200
building tree 173 of 200
building tree 174 of 200
building tree 175 of 200
building tree 176 of 200
building tree 177 of 200
building tree 178 of 200
building tree 179 of 200
building tree 180 of 200
building tree 181 of 200
building tree 182 of 200
building tree 183 of 200
building tree 184 of 200
building tree 185 of 200
building tree 186 of 200
building tree 187 of 200
building tree 188 of 200
building tree 189 of 200
building tree 190 of 200
building tree 191 of 200
building tree 192 of 200
building tree 193 of 200
building tree 194 of 200
building tree 195 of 200
building tree 196 of 200


In [ ]:
# print our macro score of the whole dataset
y_target_mine = model.predict(data_f_score)
print("\n\n\nfscore macro of our model on all data:         ", f1_score(y_true, y_target_mine, average='macro'))
print("confustion matrix: ")
print(confusion_matrix(y_true, y_target_mine))

In [ ]:
# submission
data_test_X = data_test.drop(columns=['id'])
y_target = model.predict(data_test_X)
y_target = y_target.astype(int)
print("sum: ", sum(y_target==0))

data_out = pd.DataFrame(data_test['id'].copy())
data_out.insert(1, "target", y_target, True) 
data_out.to_csv('submission.csv',index=False)

data_out